0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral.

##Importing Libraries

In [2]:
import numpy as np
import pandas as pd
from keras.layers import Input, Conv2D, LeakyReLU, Dropout, BatchNormalization, Flatten, Dense, Embedding, Reshape, Activation, Concatenate, Conv2DTranspose
from keras.models import Model
from keras.optimizers import Adam
from numpy.random import randn, randint
from matplotlib import pyplot as plt
from keras.models import save_model
from math import sqrt
from numpy import asarray
from numpy.random import randn
from keras.models import load_model

##Loading and Preprocessing the dataset

In [3]:
def load_fer2013_dataset():
    data = pd.read_csv('fer_chnge.csv')
    # Extract pixel and emotion labels
    pixels = data['pixels'].values
    emotions = data['emotion'].values
    X = []
    y = []
    for pixel_sequence, emotion in zip(pixels, emotions):
        # Parse the pixel sequence string into a list of integers
        pixel_values = [int(pixel) for pixel in pixel_sequence.split()]
        # Ensure that the pixel values array matches the expected size (48x48 = 2304 pixels)
        if len(pixel_values) != 2304:
            continue  # Skip sequences that are not 48x48
        # Reshape the pixel values into a 48x48 grayscale image (1 channel)
        image_array = np.array(pixel_values, dtype=np.uint8).reshape(48, 48, 1)
        # Normalize pixel values to range [0, 1]
        image_array = image_array.astype('float32') / 255.0
        # Append the processed image array and corresponding emotion label to the lists
        X.append(image_array)
        y.append(emotion)
    # Convert the lists to numpy arrays
    X = np.array(X)
    y = np.array(y, dtype='int32')
    return (X, y)

##Discriminator Model

In [4]:
def define_discriminator(input_shape=(48, 48, 1), n_classes=7):
    input_image = Input(shape=input_shape)
    fe = Conv2D(32, (3,3), strides=(2,2), padding='same')(input_image)
    fe = LeakyReLU(alpha=0.2)(fe)
    fe = Dropout(0.5)(fe)
    fe = Conv2D(64, (3,3),strides=(2,2), padding='same')(fe)
    fe = BatchNormalization()(fe)
    fe = LeakyReLU(alpha=0.2)(fe)
    fe = Dropout(0.5)(fe)
    fe = Conv2D(128, (3,3), strides=(2,2), padding='same')(fe)
    fe = BatchNormalization()(fe)
    fe = LeakyReLU(alpha=0.2)(fe)
    fe = Dropout(0.5)(fe)
    fe = Conv2D(256, (3,3), padding='same')(fe)
    fe = BatchNormalization()(fe)
    fe = LeakyReLU(alpha=0.2)(fe)
    fe = Dropout(0.5)(fe)
    fe = Flatten()(fe)
    out1 = Dense(1, activation='sigmoid')(fe)
    out2 = Dense(n_classes, activation='softmax')(fe)
    model = Model(input_image, [out1, out2])
    opt = Adam(learning_rate=0.0002, beta_1=0.5)
    model.compile(loss=['binary_crossentropy', 'sparse_categorical_crossentropy'], optimizer=opt)
    return model
d=define_discriminator()
d.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 48, 48, 1)]          0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 24, 24, 32)           320       ['input_1[0][0]']             
                                                                                                  
 leaky_re_lu (LeakyReLU)     (None, 24, 24, 32)           0         ['conv2d[0][0]']              
                                                                                                  
 dropout (Dropout)           (None, 24, 24, 32)           0         ['leaky_re_lu[0][0]']         
                                                                                              

##Generator Model

In [5]:
def define_generator(latent_dim, n_classes=7):
    input_label = Input(shape=(1,))
    li = Embedding(n_classes, 50)(input_label)
    n_nodes = 12 * 12
    li = Dense(n_nodes)(li)
    li = Reshape((12, 12, 1))(li)
    input_lat = Input(shape=(latent_dim,))
    n_nodes = 384 * 12 * 12
    gen = Dense(n_nodes)(input_lat)
    gen = Activation('relu')(gen)
    gen = Reshape((12, 12, 384))(gen)
    merge = Concatenate()([gen, li])
    gen = Conv2DTranspose(192, (5,5), strides=(2,2), padding='same')(merge)
    gen = BatchNormalization()(gen)
    gen = Activation('relu')(gen)
    gen = Conv2DTranspose(1, (5,5), strides=(2,2), padding='same')(gen)
    out_layer = Activation('tanh')(gen)
    model = Model([input_lat, input_label], out_layer)
    return model
g=define_discriminator()
g.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 48, 48, 1)]          0         []                            
                                                                                                  
 conv2d_4 (Conv2D)           (None, 24, 24, 32)           320       ['input_2[0][0]']             
                                                                                                  
 leaky_re_lu_4 (LeakyReLU)   (None, 24, 24, 32)           0         ['conv2d_4[0][0]']            
                                                                                                  
 dropout_4 (Dropout)         (None, 24, 24, 32)           0         ['leaky_re_lu_4[0][0]']       
                                                                                            

##GAN Model(D+G)

In [6]:
def define_gan(g_model, d_model):
    for layer in d_model.layers:
        if not isinstance(layer, BatchNormalization):
            layer.trainable = False
    gan_output = d_model(g_model.output)
    model = Model(g_model.input, gan_output)
    opt = Adam(learning_rate=0.0002, beta_1=0.5)
    model.compile(loss=['binary_crossentropy', 'sparse_categorical_crossentropy'], optimizer=opt)
    return model

##Generate fake samples

In [7]:
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples, n_classes=7):
    z_input = np.random.randn(latent_dim * n_samples)
    z_input = z_input.reshape(n_samples, latent_dim)
    labels = np.random.randint(0, n_classes, n_samples).reshape(-1, 1)
    X_fake = generator.predict([z_input, labels])
    return X_fake, labels

##Train the GAN

In [8]:
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs, n_batch):
    X_train, y_train = dataset
    bat_per_epo = 42
    for epoch in range(1, n_epochs + 1):
        print(f"Epoch {epoch}/{n_epochs}")

        for batch in range(bat_per_epo):
            ix = np.random.randint(0, X_train.shape[0], n_batch)
            X_real, labels_real = X_train[ix], y_train[ix]
            y_real = np.ones((n_batch, 1))
            _, d_r1, d_r2 = d_model.train_on_batch(X_real, [y_real, labels_real])

            X_fake, labels_fake = generate_fake_samples(g_model, latent_dim, n_batch)
            y_fake = np.zeros((n_batch, 1))
            _, d_f, d_f2 = d_model.train_on_batch(X_fake, [y_fake, labels_fake])

            z_input = np.random.randn(n_batch * latent_dim)
            z_input = z_input.reshape(n_batch, latent_dim)
            z_labels = np.random.randint(0, 7, n_batch).reshape(-1, 1)
            y_gan = np.ones((n_batch, 1))
            _, g_1, g_2 = gan_model.train_on_batch([z_input, z_labels], [y_gan, z_labels])

            print(f"  Batch {batch + 1}/{bat_per_epo}, d_real_loss: {d_r1:.4f}, d_real_acc: {d_r2:.4f}, "
                  f"d_fake_loss: {d_f:.4f}, d_fake_acc: {d_f2:.4f}, g_loss: {g_1:.4f}, g_acc: {g_2:.4f}")

        # Save generated images every few epochs
        if epoch % 100 == 0:
          examples=10
          latent_dim=100
          latent_points = np.random.randn(examples * latent_dim)
          latent_points = latent_points.reshape(examples, latent_dim)
          labels = np.random.randint(0, 7, examples).reshape(-1, 1)
          X_fake = g_model.predict([latent_points, labels])
          plt.figure(figsize=(10, 10))
          for i in range(examples):
            plt.subplot(1, examples, i + 1)
            plt.imshow(X_fake[i].reshape(48, 48), cmap='gray')
            plt.title('Label: %d' % labels[i])
            plt.axis('off')
          filename = 'generated_plot_epoch_%04d.png' % (epoch)
          plt.savefig(filename)
          plt.close()

##Run the GAN

In [9]:
def run_cgan():
    dataset = load_fer2013_dataset()
    X_train, y_train = dataset
    d_model = define_discriminator(input_shape=(48, 48, 1))
    g_model = define_generator(latent_dim=100)
    gan_model = define_gan(g_model, d_model)
    train(g_model, d_model, gan_model, (X_train, y_train), latent_dim=100, n_epochs=500, n_batch=64)
    save_model(d_model, 'discriminator_model.h5')  # Save discriminator model
    # Save generator model
    save_model(g_model, 'generator_model.h5')  # Save generator model
    # Save GAN model (including both generator and discriminator)
    save_model(gan_model, 'acgan_model.h5')  # Save CGAN model
run_cgan()

Epoch 1/500
2/2 [==============================] - 1s 433ms/step
  Batch 1/42, d_real_loss: 1.3709, d_real_acc: 2.3373, d_fake_loss: 0.7457, d_fake_acc: 2.6671, g_loss: 1.0247, g_acc: 2.8267
2/2 [==============================] - 1s 674ms/step
  Batch 2/42, d_real_loss: 1.0934, d_real_acc: 2.4909, d_fake_loss: 0.9252, d_fake_acc: 2.8687, g_loss: 0.9025, g_acc: 2.7391
2/2 [==============================] - 1s 403ms/step
  Batch 3/42, d_real_loss: 0.8922, d_real_acc: 2.5437, d_fake_loss: 1.0721, d_fake_acc: 2.5381, g_loss: 0.8822, g_acc: 2.7842
2/2 [==============================] - 1s 418ms/step
  Batch 4/42, d_real_loss: 0.8818, d_real_acc: 2.9179, d_fake_loss: 0.9610, d_fake_acc: 2.6372, g_loss: 0.8779, g_acc: 3.0088
2/2 [==============================] - 1s 696ms/step
  Batch 5/42, d_real_loss: 0.8215, d_real_acc: 2.7188, d_fake_loss: 0.9943, d_fake_acc: 2.7340, g_loss: 1.1860, g_acc: 2.6298
2/2 [==============================] - 1s 429ms/step
  Batch 6/42, d_real_loss: 0.8961, d_rea

KeyboardInterrupt: 